In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 54, 65, 66, 68, 72, 73,  0],
        [78,  1, 55, 58, 60, 54, 67,  1],
        [72, 58,  0, 54, 65, 65,  1, 73],
        [71, 57,  1, 62, 72, 67,  5, 73]], device='cuda:0')
targets:
tensor([[54, 65, 66, 68, 72, 73,  0, 57],
        [ 1, 55, 58, 60, 54, 67,  1, 73],
        [58,  0, 54, 65, 65,  1, 73, 61],
        [57,  1, 62, 72, 67,  5, 73,  1]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


c]8g'py'o.tsaX:bSiKCG)RRLdymIDtI&hJKuJ4q1FKTTB:sE32"v_Sdrz:1,2kza;!sQ2c2[2eFOlDyC wYAZrEUj,)_bS:z78;:9p?Ex[;DQ﻿X2X?(W'Ox?VD]y
8tYy3dnthLEB-V4)Od_62e l*0BB2_SIAA4s'KrP9ro,N4RL1S"OdJm7W:4ZydorPV';c:iJt;0iX7.e[eZ3DmD0
kJ
[SlQ'FhRTCjaPu:]?GIp?_0BLLwF(fSAB:BD4RB5EGPbfS﻿_ye )]nYR-!C0W:HH?_VQWd63v9Cd62CwSKv )VZFZs-W71Ko5k[
Zy,sPye K8﻿MySv:[uCCJnYFk﻿Mznt[Uj_9uj_dotSeZ&PCJOZsaK8QoJ4sEA.FCm(DJv3,]cMwGIJ﻿YR?RLv(]R*Q'oU z(R*_c,UBi5yC![rIb6MTpV)e:fhVhbwYGT_kYu]-ptgm3lc(."'Kx_ceHobMYyCmDyDyW:St
[Hm-*m383_XotO


In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.810, val loss: 4.821
step: 250, train loss: 4.753, val loss: 4.756
step: 500, train loss: 4.689, val loss: 4.675
step: 750, train loss: 4.630, val loss: 4.608
4.529431343078613


In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


bGKdMSJX?R*j.B***'oYHd﻿hbsuEPy Zsu*kwDyS,)e78g[Ld0NreW&Oss-
rsH4O
A &﻿Czpcrq'3o[[
KaWF*0O6[RIQxR*aGC87wG8g7j55BK'.nYc(oro?LiS[WzI,'*F780i.u?dzJ7Nt6edJ&gKh?I08mRf]eWnJrJIbe]8&_y7q(n6no'RV'oYId7W7﻿[*Ge490u:nYw46saGKE
49r"eV."v
g,)﻿5﻿g3DkFYIYI_(ATnT;14soExqFfrnWg&EBhe& Mj1h""DyW:,)
o!BqI-mTU.n[puh8
8egkLtwhB!BmUMTm 7e5ZLt8bSsPVQ9QYw"O1b&uL1f(WzoASg28gvzOl:((o0putkKSA5N4:ugkPBU1NPUju'.u'*
R2BrEn!u]wxJkT&'Bk5149G0h?_z;XG6e_Xbt- &:ju'CUlJPkgS'60HQ2iMjy'Fu'o4ntyQK0aG53vedKCm?RXM&.eUP"WNeM[FgbRaj
N;:U5:
